In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# --- Path Setup ---
if os.path.basename(os.getcwd()) == 'notebooks':
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
else:
    PROJECT_ROOT = os.path.abspath(os.getcwd())
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

from pipeline_config import CONFIG

# --- Directories ---
DERIV_05 = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_05_kinematics")
OUTPUT_06 = os.path.join(PROJECT_ROOT, "analysis", "step_06_plots")

os.makedirs(OUTPUT_06, exist_ok=True)

# Define plotting style
plt.style.use('seaborn-v0_8-muted')
sns.set_context("talk")

In [2]:
# Derive filename from config (synchronized with previous notebooks)
csv_filename = Path(CONFIG['current_csv']).stem
RUN_ID = csv_filename
INPUT_FILE = Path(DERIV_05) / f"{RUN_ID}__kinematics.parquet"

if not INPUT_FILE.exists():
    raise FileNotFoundError(f"Expected file not found: {INPUT_FILE}. Did you run notebook 06?")

print(f"Loading Analyzed Data: {RUN_ID}")
df = pd.read_parquet(INPUT_FILE)

FileNotFoundError: Expected file not found: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_05_kinematics\763_T2_P2_R2_Take_2025-12-25 10.51.23 AM_005__kinematics.parquet. Did you run notebook 06?

In [ ]:
def plot_joint_trajectory_3d(df, joint_name, run_id, save_dir):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Extract coordinates (assuming __px, __py, __pz columns)
    x = df[f"{joint_name}__px"]
    y = df[f"{joint_name}__py"] # Height
    z = df[f"{joint_name}__pz"]
    
    # Plot the path with a color gradient based on time
    path = ax.scatter(x, z, y, c=df['time_s'], cmap='viridis', s=2, alpha=0.6)
    
    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Z (mm)')
    ax.set_zlabel('Height Y (mm)')
    ax.set_title(f"3D Trajectory: {joint_name}\nRun: {run_id}")
    
    cbar = fig.colorbar(path, ax=ax, shrink=0.5, aspect=10)
    cbar.set_label('Time (s)')
    
    plt.savefig(os.path.join(save_dir, f"{run_id}_{joint_name}_trajectory_3d.png"))
    plt.show()

# Example: Plot Right Wrist trajectory
plot_joint_trajectory_3d(df, "RightHand", RUN_ID, OUTPUT_06)

In [ ]:
def plot_phase_plane(df, joint_angle_col, run_id, save_dir):
    plt.figure(figsize=(8, 8))
    
    angle = df[joint_angle_col]
    velocity = df[f"{joint_angle_col}_vel"]
    
    plt.plot(angle, velocity, alpha=0.7, color='teal')
    plt.axhline(0, color='black', lw=1, ls='--')
    plt.axvline(0, color='black', lw=1, ls='--')
    
    plt.title(f"Phase Plane Analysis: {joint_angle_col}")
    plt.xlabel("Angle (deg)")
    plt.ylabel("Angular Velocity (deg/s)")
    plt.grid(True, alpha=0.3)
    
    plt.savefig(os.path.join(save_dir, f"{run_id}_{joint_angle_col}_phase_plane.png"))
    plt.show()

# Example: Analyze Knee coordination
plot_phase_plane(df, "LeftKnee_Angle_X", RUN_ID, OUTPUT_06)

In [ ]:
# Select major flexion angles for correlation
angle_cols = [c for c in df.columns if "_Angle_X" in c and "Toe" not in c]

corr = df[angle_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=False, cmap='RdBu_r', center=0, square=True)
plt.title(f"Joint Coordination Heatmap\nRun: {RUN_ID}")
plt.savefig(os.path.join(OUTPUT_06, f"{RUN_ID}_coordination_heatmap.png"))
plt.show()

In [ ]:
def export_summary_stats(df, run_id, save_dir):
    # Select only numeric columns we want to summarize
    cols = [c for c in df.columns if "_Angle" in c]
    
    summary = df[cols].agg(['mean', 'std', 'min', 'max']).T
    
    # Calculate Peak-to-Peak Range
    summary['range'] = summary['max'] - summary['min']
    
    out_path = os.path.join(save_dir, f"{run_id}__kinematics_summary.csv")
    summary.to_csv(out_path)
    print(f"Summary metrics exported to: {out_path}")
    return summary

summary_stats = export_summary_stats(df, RUN_ID, OUTPUT_06)
display(summary_stats.head(20))